In [1]:
from dataclasses import dataclass

In [6]:
import torch
import torch.nn as nn

In [7]:
import torch.nn.functional as F

In [1]:
from transformers import AutoModel, AutoTokenizer

In [4]:
!modelscope login --token 82b2e151-12af-4f0d-8f99-fba9c523e113

In [ ]:
from modelscope.hub.api import HubApi
import os

os.environ['all_proxy'] = ''
model_path = 'outputs/Qwen-0.5B-GRPO'
repo_id = 'learn2pro/Qwen-0.5B-GRPO'

api = HubApi()
api.login('82b2e151-12af-4f0d-8f99-fba9c523e113')

api.push_model(
    model_dir=model_path,
    model_id=repo_id
)

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
model_path = 'outputs/Qwen-0.5B-GRPO'
repo_id = 'learn2pro/Qwen-0.5B-GRPO'

# 加载模型和分词器
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 推送到 Hugging Face Hub
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
repo_id = 'learn2pro/Qwen-3B-GRPO'

# 加载模型和分词器
model = AutoModelForCausalLM.from_pretrained(repo_id)
model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained(repo_id)
print(tokenizer.chat_template)

In [ ]:
from vllm import LLM
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

from transformers import TextStreamer
import torch

SYSTEM_PROMPT = """
Responde in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

def generate_with_stream(input_text):
    print(f"\n输入: \n{input_text}")
    print("\n输出:")

    prompts = [
        {"role":"system", "content": SYSTEM_PROMPT},
        {"role":"user", "content":input_text},
    ]
    chats = tokenizer.apply_chat_template(prompts, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(chats, return_tensors="pt").to("cuda")
    streamer = TextStreamer(tokenizer)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=512,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            streamer=streamer
        )
    
    # 完整结果
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)


# 使用
# input_text = "1+3*9/2+1.2+(1+2*3)=?"
input_text = '''
A very special island is inhabited only by knights and knaves. Knights always tell
the truth, and knaves always lie. You meet 2 inhabitants: Zoey, and Oliver. Zoey remarked,
"Oliver is not a knight". Oliver stated, "Oliver is a knight if and only if Zoey is a knave". So
who is a knight and who is a knave?
'''
generate_with_stream(input_text)

In [ ]:
from evalscope import TaskConfig, run_task
import os, httpx
os.environ['all_proxy'] = '127.0.0.1:7890'

# task_config = TaskConfig(
#     api_url='http://127.0.0.1:8000/v1/chat/completions',  # Inference service address
#     model='gpt2',  # Model name (must match the deployed model name)
#     eval_type='service',  # Evaluation type, SERVICE indicates evaluating the inference service
#     datasets=['math_500'],  # Dataset name
#     dataset_args={'math_500': {'few_shot_num': 0, 'subset_list': ['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5']}},  # Dataset parameters
#     eval_batch_size=32,  # Number of concurrent requests
#     generation_config={
#         'max_tokens': 20000,  # Maximum number of tokens to generate; suggested to set a high value to avoid truncation
#         'temperature': 0.6,  # Sampling temperature (recommended value from deepseek)
#         'top_p': 0.95,  # Top-p sampling (recommended value from deepseek)
#         'n': 1,  # Number of responses generated for each request
#     },
# )

task_config = {
    'model': 'learn2pro/Qwen-3B-GRPO',
    'datasets': ['gsm8k', 'arc'],
    'limit': 5
}

run_task(task_config)

In [7]:
import torch
torch.randn(5).unsqueeze(0).repeat(5,1)

tensor([[ 0.5349,  0.8094,  1.1103, -1.6898, -0.9890],
        [ 0.5349,  0.8094,  1.1103, -1.6898, -0.9890],
        [ 0.5349,  0.8094,  1.1103, -1.6898, -0.9890],
        [ 0.5349,  0.8094,  1.1103, -1.6898, -0.9890],
        [ 0.5349,  0.8094,  1.1103, -1.6898, -0.9890]])